In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_movie = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
df_credit = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
movies = df_movie.merge(df_credit, on='title')

In [3]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [4]:
important_Columns = ['genres', 'id', 'keywords', 'overview', 'popularity', 'release_date', 'title', 'cast', 'crew']

In [5]:
movies = movies[important_Columns]

In [6]:
movies.isnull().sum()

genres          0
id              0
keywords        0
overview        3
popularity      0
release_date    1
title           0
cast            0
crew            0
dtype: int64

In [7]:
movies.dropna(inplace= True)

In [8]:
movies.duplicated().sum()

0

In [9]:
import ast 


def convert(obj):
    temp = []
    for i in ast.literal_eval(obj):
        # converted string into list of dict's
        temp.append(i['name'])
    return temp

In [10]:
movies['genres'] = movies['genres'].apply(convert)

In [11]:
movies['keywords'] = movies['keywords'].apply(convert)

In [12]:
def convert_cast(obj):
    temp = []
    counter = 0
    for i in ast.literal_eval(obj):
        # converted string into list of dict's
        if counter  != 3:
            temp.append(i['name'])
            counter+=1
        else:
            break
    return temp

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
def convert_crew(obj):
    for i in ast.literal_eval(obj):
        if i['department'] == 'Directing':
            return [i['name']]

In [15]:
movies['crew'] = movies['crew'].apply(convert_crew)

In [16]:
movies['popularity'] = movies['popularity'].astype(int)


In [17]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4805 entries, 0 to 4808
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        4805 non-null   object
 1   id            4805 non-null   int64 
 2   keywords      4805 non-null   object
 3   overview      4805 non-null   object
 4   popularity    4805 non-null   int64 
 5   release_date  4805 non-null   object
 6   title         4805 non-null   object
 7   cast          4805 non-null   object
 8   crew          4776 non-null   object
dtypes: int64(2), object(7)
memory usage: 375.4+ KB


In [18]:
movies.dropna(inplace = True)
movies.isnull().sum()

genres          0
id              0
keywords        0
overview        0
popularity      0
release_date    0
title           0
cast            0
crew            0
dtype: int64

In [19]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [20]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [21]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [22]:
result_df = movies[['id', 'title', 'tags']]

In [23]:
result_df['tags'] = result_df['tags'].apply(lambda x: " ".join(x))
result_df['tags'] = result_df['tags'].apply(lambda x: x.lower())

/tmp/ipykernel_30/1675263045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['tags'] = result_df['tags'].apply(lambda x: " ".join(x))
/tmp/ipykernel_30/1675263045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['tags'] = result_df['tags'].apply(lambda x: x.lower())


In [24]:
from nltk.stem.porter import PorterStemmer

Stemmer = PorterStemmer()
def Stemm(text):
    y = []

    for i in text.split():
        y.append(Stemmer.stem(i))
    return " ".join(y)

result_df['tags'].apply(Stemm)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4776, dtype: object

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 5000, stop_words='english')

In [39]:
vectors = cv.fit_transform(result_df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.08980265, 0.05986843, ..., 0.0248452 , 0.02635231,
        0.        ],
       [0.08980265, 1.        , 0.06451613, ..., 0.02677398, 0.        ,
        0.        ],
       [0.05986843, 0.06451613, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.0248452 , 0.02677398, 0.02677398, ..., 1.        , 0.07071068,
        0.04836508],
       [0.02635231, 0.        , 0.        , ..., 0.07071068, 1.        ,
        0.05129892],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05129892,
        1.        ]])

In [50]:
def recommend(movie):
    index = result_df[result_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(result_df.iloc[i[0]].title)
        

In [54]:
recommend('Bloody Sunday')

One True Thing
The Exploding Girl
The New World
Marvin's Room
The Heart of Me


In [55]:
import pickle

In [ ]:
pickle.dump(result_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))